In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
from pydrake.all import (
    AddDefaultVisualization,
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    LoadModelDirectives,
    LoadModelDirectivesFromString,
    Parser,
    ProcessModelDirectives,
    RigidTransform,
    RollPitchYaw,
    Simulator,
    StartMeshcat,
    FindResourceOrThrow,
    MultibodyPlant,
    MeshcatVisualizer,
    InverseDynamicsController,
    PassThrough,
    Demultiplexer,
    StateInterpolatorWithDiscreteDerivative,
    MeshcatPoseSliders,
    ConstantVectorSource,
    JointSliders,
    Multiplexer,
    RigidTransform,
    Integrator
)
import manipulation
from manipulation.station import LoadScenario, MakeHardwareStation
from manipulation.utils import ConfigureParser, RenderDiagram

import sys

sys.path.append("../")

from src.station import  MakePandaManipulationStation
from src.diff_ik import TrajectoryPlanner, PandaDiffIKController

In [3]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7002


In [22]:
# description of robot
robot_directives = """
directives:
    - add_model:
        name: panda_arm
        file: package://drake_models/franka_description/urdf/panda_arm.urdf
        default_joint_positions:
            panda_joint1: [-1.57]
            panda_joint2: [0.1]
            panda_joint3: [0]
            panda_joint4: [-1.2]
            panda_joint5: [0]
            panda_joint6: [ 1.6]
            panda_joint7: [0]
    - add_weld:
        parent: world
        child: panda_arm::panda_link0
    - add_model:
        name: panda_hand
        file: package://drake_models/franka_description/urdf/panda_hand.urdf
    - add_weld:
        parent: panda_arm::panda_link8
        child: panda_hand::panda_hand
        X_PC:
            translation: [0, 0, 0]
            rotation: !Rpy { deg: [0, 0, -45] }
"""

# description of objects in env
env_directives = """
directives:
    # - add_model:
    #     name: floor
    #     file: package://manipulation/floor.sdf
    # - add_weld:
    #     parent: world
    #     child: floor::box
    #     X_PC:
    #         translation: [0, 0, -0.05]

    - add_model:
        name: foam_brick1
        file: package://manipulation/hydro/061_foam_brick.sdf
    - add_weld:
        parent: world
        child: foam_brick1::base_link
        X_PC:
            translation: [0.0003675628234776587, -0.46157274678443827, 0.8213981598432528]

    - add_model:
        name: foam_brick2
        file: package://manipulation/hydro/061_foam_brick.sdf
    - add_weld:
        parent: world
        child: foam_brick2::base_link
        X_PC:
            translation: [0.0003675628234776587, -0.46157274678443827, 0.75]
"""

In [23]:
builder = DiagramBuilder()

# add panda manipulation station
# station is a system consisting of subsytems ie. multi body plant, scene graphs, controllers ... etc
station = MakePandaManipulationStation(
    robot_directives=robot_directives,
    env_directives=env_directives,
    meshcat=meshcat,
    time_step=1e-4
)

builder.AddSystem(station)

plant = station.GetSubsystemByName("plant")
panda_arm = plant.GetModelInstanceByName("panda_arm")
panda_hand = plant.GetModelInstanceByName("panda_hand")

num_panda_arm_positions = plant.num_positions(panda_arm)
num_panda_hand_positions = plant.num_positions(panda_hand)


meshcat.DeleteAddedControls()

# create a joint sliders system

# get initial pose of end effector
context = plant.CreateDefaultContext()
plant.SetPositions(context, panda_arm, [-1.57, 0.1, 0, -1.2, 0, 1.6, 0])
initial_pose = plant.EvalBodyPoseInWorld(context, plant.GetBodyByName("panda_link6"))

# print("initial_pose: ", initial_pose)
initial_pose = RigidTransform(R=initial_pose.rotation(), p=[0, -0.46, 0.9])


pose_sliders = builder.AddSystem(MeshcatPoseSliders(meshcat, initial_pose=initial_pose))
trajectory_planner = builder.AddSystem(TrajectoryPlanner(plant=plant))
diff_ik = builder.AddSystem(PandaDiffIKController(plant=plant))
integrator = builder.AddSystem(Integrator(7))

builder.Connect(pose_sliders.get_output_port(0), trajectory_planner.GetInputPort("goal_pose"))
builder.Connect(trajectory_planner.GetOutputPort("desired_spatial_velocity"), diff_ik.GetInputPort("spatial_velocity"))
builder.Connect(station.GetOutputPort("panda_arm.position_estimated"), diff_ik.GetInputPort("q"))

builder.Connect(diff_ik.get_output_port(0), integrator.get_input_port(0))
builder.Connect(integrator.get_output_port(0), station.GetInputPort("panda_arm.position"))

diagram = builder.Build()
diagram.set_name("PandaFunzo")

context = diagram.CreateDefaultContext()
simulator = Simulator(diagram, context)
simulator.set_target_realtime_rate(1.0)

integrator.set_integral_value(
    integrator.GetMyContextFromRoot(context), 
    plant.GetPositions(plant.GetMyContextFromRoot(context), panda_arm)
)


meshcat.StartRecording()
# simulator.AdvanceTo(np.inf)
simulator.AdvanceTo(15.0)
meshcat.PublishRecording()

goal chacned
